In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]


2020-11-08 19:15:48,313 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-08 19:15:48,314 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [4]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2020-11-08 19:15:48,319 : INFO : using serial LSI version on this node
2020-11-08 19:15:48,320 : INFO : updating model with new documents
2020-11-08 19:15:48,320 : INFO : preparing a new chunk of documents
2020-11-08 19:15:48,322 : INFO : using 100 extra samples and 2 power iterations
2020-11-08 19:15:48,322 : INFO : 1st phase: constructing (12, 102) action matrix
2020-11-08 19:15:48,324 : INFO : orthonormalizing (12, 102) action matrix
2020-11-08 19:15:48,432 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2020-11-08 19:15:48,440 : INFO : computing the final decomposition
2020-11-08 19:15:48,440 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2020-11-08 19:15:48,443 : INFO : processed documents up to #9
2020-11-08 19:15:48,444 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2020-11-08 19:15:48,446 : INFO : topic #1(2

In [5]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, 0.4618210045327159), (1, 0.0700276652789994)]


In [6]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

2020-11-08 19:15:48,459 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2020-11-08 19:15:48,461 : INFO : creating matrix with 9 documents and 2 features
2020-11-08 19:15:48,463 : INFO : saving MatrixSimilarity object under /tmp/deerwester.index, separately None
2020-11-08 19:15:48,464 : INFO : saved /tmp/deerwester.index
2020-11-08 19:15:48,466 : INFO : loading MatrixSimilarity object from /tmp/deerwester.index
2020-11-08 19:15:48,466 : INFO : loaded /tmp/deerwester.index


In [7]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [8]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sims):
    print(s, documents[i])

(2, 0.9984453) Human machine interface for lab abc computer applications
(0, 0.998093) A survey of user opinion of computer system response time
(3, 0.9865886) The EPS user interface management system
(1, 0.93748635) System and human system engineering testing of EPS
(4, 0.90755945) Relation of user perceived response time to error measurement
(8, 0.050041765) The generation of random binary unordered trees
(7, -0.09879464) The intersection graph of paths in trees
(6, -0.10639259) Graph minors IV Widths of trees and well quasi ordering
(5, -0.12416792) Graph minors A survey
